<a href="https://colab.research.google.com/github/nikatnguyen/Project4/blob/main/anika_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [3]:
#Import dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data class notebooks/clean_df.csv')
df.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Insufficient_Weight,Normal_Weight,...,yes.3,Always.1,Frequently.1,Sometimes.1,no.5,Automobile,Bike,Motorbike,Public_Transportation,Walking
0,21.0,1.62,64.0,2.0,3.0,2.0,0.0,1.0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,21.0,1.52,56.0,3.0,3.0,3.0,3.0,0.0,0,1,...,1,0,0,1,0,0,0,0,1,0
2,23.0,1.80,77.0,2.0,3.0,2.0,2.0,1.0,0,1,...,0,0,1,0,0,0,0,0,1,0
3,27.0,1.80,87.0,3.0,3.0,2.0,2.0,0.0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,22.0,1.78,89.8,2.0,1.0,2.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [5]:
#Split data set into features and targets
y = df[['Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']].values
X = df.drop(columns=['Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ATTEMPT 1

In [7]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 0.7399 - accuracy: 0.1592
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.1592
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6660 - accuracy: 0.1592
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6469 - accuracy: 0.1592
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6310 - accuracy: 0.1592
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6168 - accuracy: 0.1592
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6038 - accuracy: 0.1592
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5918 - accuracy: 0.1592
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5805 - accuracy: 0.1592
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5699 - accuracy: 0.1592
Epoch 11/50
50/50 [

ATTEMPT 2

In [8]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 2s 3ms/step - loss: 0.5202 - accuracy: 0.1586
Epoch 2/50
50/50 [==============================] - 0s 3ms/step - loss: 0.4297 - accuracy: 0.1781
Epoch 3/50
50/50 [==============================] - 0s 4ms/step - loss: 0.3827 - accuracy: 0.3203
Epoch 4/50
50/50 [==============================] - 0s 4ms/step - loss: 0.3478 - accuracy: 0.4037
Epoch 5/50
50/50 [==============================] - 0s 4ms/step - loss: 0.3144 - accuracy: 0.4725
Epoch 6/50
50/50 [==============================] - 0s 3ms/step - loss: 0.2818 - accuracy: 0.5016
Epoch 7/50
50/50 [==============================] - 0s 4ms/step - loss: 0.2518 - accuracy: 0.5218
Epoch 8/50
50/50 [==============================] - 0s 4ms/step - loss: 0.2258 - accuracy: 0.5193
Epoch 9/50
50/50 [==============================] - 0s 5ms/step - loss: 0.2036 - accuracy: 0.5313
Epoch 10/50
50/50 [==============================] - 0s 4ms/step - loss: 0.1844 - accuracy: 0.5439
Epoch 11/50
50/50 [

ATTEMPT 3

In [9]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 0.7058 - accuracy: 0.4119
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5349 - accuracy: 0.5692
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4438 - accuracy: 0.6785
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3948 - accuracy: 0.7081
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3631 - accuracy: 0.7498
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3376 - accuracy: 0.7979
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3149 - accuracy: 0.8099
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2947 - accuracy: 0.8225
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2769 - accuracy: 0.8231
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2611 - accuracy: 0.8370
Epoch 11/50
50/50 [

ATTEMPT 4

In [11]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 1.9313 - accuracy: 0.1440
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7831 - accuracy: 0.1440
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7524 - accuracy: 0.1440
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7352 - accuracy: 0.1440
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7222 - accuracy: 0.1440
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7119 - accuracy: 0.1440
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 1.7033 - accuracy: 0.1440
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 1.6972 - accuracy: 0.1440
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 1.6943 - accuracy: 0.1440
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 1.6905 - accuracy: 0.1440
Epoch 11/50
50/50 [